# Basics of array

In [1]:
import numpy as np

## 1. numpy가 array를 저장하는 방식

메모리는 1차원적으로 정보를 저장한다.
2차원 nxn array를 0번째부터 n^2-1번째의 메모리 공간에 저장하려면 다음과 같은 규칙으로 값을 저장할 수 있을 것이다.

$$ memory[x+n*y] = array[y,x] $$
$$ or $$
$$ memory[x+n*y] = array[x,y] $$

위의 방식을 row-major array라고 하고, 아래 방식을 column-major array 방식이라고 한다.
python의 numpy는 row-major array 방식을 이용하며, matlab과 julia는 column-major array 방식을 취한다.


In [2]:
x = np.arange(6)
# 1-dim indexing
print(x[0],x[1],x[2],x[3],x[4],x[5])
x = x.reshape(3,2)
# row-major indexing
print(x[0,0], x[0,1], x[1,0], x[1,1], x[2,0], x[2,1])

0 1 2 3 4 5
0 1 2 3 4 5


이로 인해 모든 element를 접근하려고 할 때 numpy는 row-major indexing을 사용해야 더 최적의 결과를 얻을 수 있다.

아래에 matlab, julia는 column-major indexing이 더 빠른 것을 확인할 수 있다.

**matlab**
```
x = ones(1024,1024);
tic()
for i = 1:1024
    for j = 1:1024
        x(i,j) = rand();
    end
end
toc() % row-majored, 19.785ms

x = ones(1024,1024);
tic()
for i = 1:1024
    for j = 1:1024
        x(j,i) = rand();
    end
end
toc() % column-majored, 19.380ms

```

**julia**
```
using BenchmarkTools

function row_major()
    x = ones(1024,1024)
    for i = 1:1024
        for j = 1:1024
            x[i,j] = rand();
        end
    end
    return x
end

function col_major()
    x = ones(1024,1024)
    for i = 1:1024
        for j = 1:1024
            x[j,i] = rand();
        end
    end
    return x
end

@benchmark row_major() # median = 5.283ms
@benchamrk col_major() # median = 4.130ms
```

In [3]:
def row_major(n):
    x = np.ones((n,n))
    for i in range(n):
        for j in range(n):
            x[i,j] = np.random.rand()

def col_major(n):
    x = np.ones((n,n))
    for i in range(n):
        for j in range(n):
            x[j,i] = np.random.rand()


In [4]:
%timeit row_major(1024) # n = 1024, 39 s ± 71.8 ms per loop
%timeit col_major(1024) # n = 1024, 39.9 s ± 232 ms per loop

5.02 s ± 11.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


KeyboardInterrupt: 

## 2. Single instruction multiple data(SIMD): 하나의 명령어로 여러 개의 값 처리하기

SIMD 명령어는 하나의 명령어로 여러 값을 처리하는 방식으로 각 계산이 서로의 결과에 전혀 영향을 미치는 않을 때 많이 사용한다.
가령 size n인 두 array를 더하는 것은 총 n번의 덧셈이 필요하다.
이 경우 SIMD 명령어를 사용하여 연산을 한번에 진행할 수 있다. 예시로는 x86-64의 AVX가 있다.

numpy는 추상화를 통해 array간 연산을 자동으로 SIMD 명령어로 수행한다.

In [ ]:
%timeit np.ones((1024,1024)) + np.random.rand(1024,1024) # n = 1024, 940 ms ± 19.4 ms

940 ms ± 19.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## 3. Broadcasting rule: 서로 다른 모양의 array 연산하기

때론 서로 다른 array를 더하고 싶은 상황이 온다.

예를 들어 256*256*3 의 RGB image에서 red 는 1만큼 낮추고, green 은 2만큼 올리고, blue는 3만큼 올리고 싶다고 해보자. 이 경우 [-1,2,3] 의 array를 256*256개가 있으면 가능하지만, 이는 쓸데없는 메모리 용량을 잡아먹고, 각 pixel에 for문을 사용하면서 [-1,2,3]을 더하는 것은 SIMD를 온전히 활용할 수 없게 한다.

이런 문제를 해결하기 위해 numpy는 broadcast를 지원한다. [참고](https://numpy.org/doc/stable/user/basics.broadcasting.html)

<case 1>

scalar + array: n x m array와 scalar를 더하려고 하면 scalar는 n x m array로 broadcasting되어 각 array element에 더해진다.

<case 2>

(n x m x k array1) + (n x m x 1 array2): array1의 각 0:n x 0:m x i 슬라이스마다 array2를 더한다.

In [ ]:
x = np.arange(4).reshape(1,4)
z = np.ones((3,4))

x*z

array([[0., 1., 2., 3.],
       [0., 1., 2., 3.],
       [0., 1., 2., 3.]])